# 가치평가 계산

In [51]:
import collector
import pandas as pd
from pykrx import stock
import FinanceDataReader as fdr
import numpy as np

In [85]:
def loadJibaeTables(data, years, main):
    jibaeyear = main.maria.showData('select * from value_jibae_{}_12'.format(years[3])).drop('FS', axis=1)
    jibaebf1year = main.maria.showData('select * from value_jibae_{}_12'.format(years[2])).drop('FS', axis=1)
    jibaeyear_e = main.maria.showData('select * from value_jibae_{}_12_e'.format(years[3])).drop('FS', axis=1)
    data['jibae_{}_12'.format(years[3])] = pd.concat([jibaeyear, jibaebf1year, jibaeyear_e]).drop_duplicates(subset='종목코드', keep='first').reset_index(drop=True)
    
def loadRoeTables(data, years, main):
    roeyear_e = main.maria.showData('select * from value_roe_{}_12_e'.format(years[3]+1)).drop('FS', axis=1)
    roeyear = main.maria.showData('select * from value_roe_{}_12'.format(years[3])).rename(columns={'ROE': 'ROE_{}_12'.format(years[3])}).drop('FS', axis=1)
    roebf1year = main.maria.showData('select * from value_roe_{}_12'.format(years[2])).rename(columns={'ROE': 'ROE_{}_12'.format(years[2])}).drop('FS', axis=1)
    roebf2year = main.maria.showData('select * from value_roe_{}_12'.format(years[1])).rename(columns={'ROE': 'ROE_{}_12'.format(years[1])}).drop('FS', axis=1)
    roebf3year = main.maria.showData('select * from value_roe_{}_12'.format(years[0])).rename(columns={'ROE': 'ROE_{}_12'.format(years[0])}).drop('FS', axis=1)                                                                                                   
    # roe 가중평균 계산
#     roe_tmp = pd.merge(roeyear, roebf1year, on=['종목코드'])
#     roe_tmp = pd.merge(roe_tmp, roebf2year, on=['종목코드'])
#     cal_roeyear = pd.DataFrame()
#     cal_roeyear['종목코드'] = roe_tmp['종목코드']
#     cal_roeyear['ROE'] = (roe_tmp['ROE_{}_12'.format(years[3])]*3 + roe_tmp['ROE_{}_12'.format(years[2])]*2 + roe_tmp['ROE_{}_12'.format(years[1])]) / 6        
#     roe_tmp = pd.merge(roebf1year, roebf2year, on=['종목코드'])                                 
#     roe_tmp = pd.merge(roe_tmp, roebf3year, on=['종목코드'])
#     cal_roebfyear = pd.DataFrame()
#     cal_roebfyear['종목코드'] = roe_tmp['종목코드']
#     cal_roebfyear['ROE'] = (roe_tmp['ROE_{}_12'.format(years[2])]*3 + roe_tmp['ROE_{}_12'.format(years[1])]*2 + roe_tmp['ROE_{}_12'.format(years[0])]) / 6

# drop_duplicates를 이용하여 가장 최신 데이터를 선정
#    data['roe_{}_12'.format(years[3]+1)] = pd.concat([roeyear_e, cal_roeyear, cal_roebfyear]).drop_duplicates(subset='종목코드', keep='first').reset_index(drop=True)
    
    # 가중평균이 아닌 최신데이터 사용
    roeyear = main.maria.showData('select * from value_roe_{}_12'.format(years[3])).drop('FS', axis=1)
    roebf1year = main.maria.showData('select * from value_roe_{}_12'.format(years[2])).drop('FS', axis=1)
    data['roe_{}_12'.format(years[3]+1)] = pd.concat([roeyear_e, roeyear, roebf1year]).drop_duplicates(subset='종목코드', keep='first').reset_index(drop=True)
    data['roe_{}_12'.format(years[3]+1)]['ROE'] = data['roe_{}_12'.format(years[3]+1)]['ROE'] / 100
    
def loadTables(data, years, selected_qt, main):
    if selected_qt != '12':
        tmp_years = years[:-1]
    else:
        tmp_years = years
    for y in tmp_years:
        data['fnguide_{}_12'.format(y)] = main.maria.showData('select * from value_fnguide_{}_12'.format(y)).drop('FS', axis=1)
        data['signal_{}_12'.format(y)] = main.maria.showData('select * from value_signal_{}_12'.format(y)).drop('FS', axis=1)
        data['dart_jb_{}_12'.format(y)] = main.maria.showData('select * from value_dart_jb_{}_12'.format(y)).drop('FS', axis=1)
        data['dart_mh_{}_12'.format(y)] = main.maria.showData('select * from value_dart_mh_{}_12'.format(y)).drop('FS', axis=1)
        data['dart_dg_{}_12'.format(y)] = main.maria.showData('select * from value_dart_dg_{}_12'.format(y)).drop('FS', axis=1)
        data['stocks_{}_12'.format(y)] = main.maria.showData('select * from value_stocks_{}_12'.format(y))
        
    if selected_qt != '12':
        for y in years:
            data['dart_dg_{}_{}'.format(y, selected_qt)] = main.maria.showData('select * from value_dart_dg_{}_{}'.format(y, selected_qt)).drop('FS', axis=1)
            data['stocks_{}_{}'.format(y, selected_qt)] = main.maria.showData('select * from value_stocks_{}_{}'.format(y, selected_qt))

        # 사업보고서 공시 전에는 전년도 데이터 사용
        data['fnguide_{}_12'.format(years[3])] = main.maria.showData('select * from value_fnguide_{}_12'.format(years[2])).drop('FS', axis=1)
        data['signal_{}_12'.format(years[3])] = main.maria.showData('select * from value_signal_{}_12'.format(years[2])).drop('FS', axis=1)
        data['dart_jb_{}_12'.format(years[3])] = main.maria.showData('select * from value_dart_jb_{}_12'.format(years[2])).drop('FS', axis=1)
        data['dart_mh_{}_12'.format(years[3])] = main.maria.showData('select * from value_dart_mh_{}_12'.format(years[2])).drop('FS', axis=1)
        # 주식 수는 최신 데이터
        data['stocks_{}_12'.format(years[3])] = main.maria.showData('select * from value_stocks_{}_12'.format(years[3]))
    loadJibaeTables(data, years, main)
    loadRoeTables(data, years, main)
    
def modifyColNames(data):
    tables = data.keys()
    for t in tables:
        cols = data[t].columns[1:]
        data[t].columns = ['종목코드'] + [c+t[-8:] for c in cols]
        
def mergeTables(data):
    tables = list(data.keys())
    df = data[tables[0]]
    for t in tables[1:]:
        df = pd.merge(df, data[t]).reset_index(drop=True)
    return df

def calDg(df, years, selected_qt):
    tmp = df[['당기순이익_{}_{}'.format(years[2], selected_qt), '당기순이익_{}_{}'.format(years[1], selected_qt), '당기순이익_{}_{}'.format(years[0], selected_qt)]]
    tmp['rate1'] = (tmp['당기순이익_{}_{}'.format(years[2], selected_qt)] - tmp['당기순이익_{}_{}'.format(years[1], selected_qt)]) / tmp['당기순이익_{}_{}'.format(years[1], selected_qt)]
    tmp['rate2'] = (tmp['당기순이익_{}_{}'.format(years[1], selected_qt)] - tmp['당기순이익_{}_{}'.format(years[0], selected_qt)]) / tmp['당기순이익_{}_{}'.format(years[0], selected_qt)]
    df['당기순이익_{}_12'.format(years[3])] = df['당기순이익_{}_12'.format(years[2])] * (1 + (tmp['rate1'] + tmp['rate2']) / 2)
    return df

def calEPS(df, years):
    for y in years:
        df['EPS_{}_12'.format(y)] = df['당기순이익_{}_12'.format(y)] / (df['보통주_{}_12'.format(y)] + df['우선주_{}_12'.format(y)])
    return df

def mergePrice(df, std_date):
    kospi = pd.DataFrame(stock.get_market_ohlcv_by_ticker(std_date)['종가'])
    kospi['Size'] = [0] * kospi.shape[0]
    kosdaq = pd.DataFrame(stock.get_market_ohlcv_by_ticker(std_date, market='KOSDAQ')['종가'])
    kosdaq['Size'] = [1] * kosdaq.shape[0]
    konex = pd.DataFrame(stock.get_market_ohlcv_by_ticker(std_date, market='KONEX')['종가'])
    konex['Size'] = [2] * konex.shape[0]
    price = pd.concat([kospi, kosdaq, konex])
    price = price.reset_index(drop=False).rename(columns={'티커': '종목코드', '종가': '주식가격'})
    df = pd.merge(df, price, on='종목코드')
    return df

def calEvaluation(df, years):
    df['순자산가치'] = (df['자본_{}_12'.format(years[3])] - df['무형자산_{}_12'.format(years[3])]) / (df['보통주_{}_12'.format(years[3])] + df['우선주_{}_12'.format(years[3])])
    df['순손익가치'] = ((df['EPS_{}_12'.format(years[1])] + df['EPS_{}_12'.format(years[2])]*2 + df['EPS_{}_12'.format(years[3])]*3)/6)*10
    df['본질가치'] = (df['순자산가치'] + (df['순손익가치']*1.5))/2.5
    df['보충적가치'] = (df['순손익가치']*2 + df['순자산가치']*3) / 5
    df['주당기업가치'] = ((df['보통주_{}_12'.format(years[3])] + df['우선주_{}_12'.format(years[3])])*df['주식가격']+df['단기차입금_{}_12'.format(years[3])]+df['장기차입금_{}_12'.format(years[3])]-df['기말현금및현금성자산_{}_12'.format(years[3])])/(df['보통주_{}_12'.format(years[3])] + df['우선주_{}_12'.format(years[3])])
    df['주당가치분석'] = (df['본질가치'] + df['주당기업가치']) / 2
    # 요구 수익률 파싱
    df['요구수익률'] = [getYogoo(main)] * df.shape[0]
    # 적정주가
    df['기업가치'] = df['지배주주지분_{}_12'.format(years[3])] + df['지배주주지분_{}_12'.format(years[3])]*(df['ROE_{}_12'.format(years[3]+1)] - df['요구수익률']) * (0.8 / (1 + df['요구수익률']-0.8))
    df['적정주가'] = df['기업가치'] / (df['보통주_{}_12'.format(years[3])] + df['우선주_{}_12'.format(years[3])] - df['자사주_{}_12'.format(years[3])])

    return df

def getYogoo(main):
    main._setChrome()
    url = 'https://www.kisrating.com/ratingsStatistics/statics_spread.do'
    main.driver.get(url)
    html = main.driver.page_source
    tmp = pd.read_html(html)
    tmp = tmp[0]
    return (tmp[tmp['구분'] == 'BBB-']['5년'].iloc[0]) / 100

def calScores(df):
    data = {}
    data['profit'] = ['자기자본수익률', '매출영업이익률', '매출액순이익률', '총자산이익률']
    수익성점수 = [[4, 2, 0], [8, 4, 0], [13, 7, 0]]
    수익성임계치 = [[25, 10], [5, 3], [5, 3], [5, 3]]
    
    data['grow'] = ['매출액증가율', '영업이익증가율', '당기순이익증가율', '자산총계증가율']
    성장성점수 = 수익성점수
    
    data['stable'] = ['유동비율', '부채비율', '자기자본비율']
    안정성점수1 = [[[6, 3, 0], [11, 6, 0], [18, 9, 0]], [[0, 3, 6], [0, 6, 11], [0, 9, 18]]]
    안정성점수2 = [[5, 0], [10, 0], [15, 0]]
    
    
    data['market'] = ['EPS', '주당순자산가치', '주당매출액']
    시장가치비율점수 = [[50, 40, 30, 10], [25, 20, 15, 5], [25, 20, 15, 5]]

    for y in years:
        df['자기자본수익률_{}_12'.format(y)] = (df['당기순이익_{}_12'.format(y)] / df['자본_{}_12'.format(y)]) * 100
        df['매출영업이익률_{}_12'.format(y)] = (df['영업이익_{}_12'.format(y)] / df['매출액_{}_12'.format(y)]) * 100
        df['매출액순이익률_{}_12'.format(y)] = (df['당기순이익_{}_12'.format(y)] / df['매출액_{}_12'.format(y)]) * 100
        df['총자산이익률_{}_12'.format(y)] = (df['당기순이익_{}_12'.format(y)] / df['자산_{}_12'.format(y)]) * 100
        df['유동비율_{}_12'.format(y)] = (df['유동자산_{}_12'.format(y)] / df['유동부채_{}_12'.format(y)]) * 100
        df['부채비율_{}_12'.format(y)] = (df['부채_{}_12'.format(y)] / df['자본_{}_12'.format(y)]) * 100
        df['자기자본비율_{}_12'.format(y)] = (df['자본_{}_12'.format(y)] / df['자산_{}_12'.format(y)]) * 100
        df['주당순자산가치_{}_12'.format(y)] = (df['자본_{}_12'.format(y)] / (df['보통주_{}_12'.format(y)] + df['우선주_{}_12'.format(y)])) * 100
        df['주당매출액_{}_12'.format(y)] = (df['매출액_{}_12'.format(y)] / (df['보통주_{}_12'.format(y)] + df['우선주_{}_12'.format(y)])) * 100
    
    for i, y in enumerate(years[1:]):
        df['매출액증가율_{}_12'.format(y)] = ((df['매출액_{}_12'.format(y)] / df['매출액_{}_12'.format(y-1)]) * 100) - 100
        df['영업이익증가율_{}_12'.format(y)] = ((df['영업이익_{}_12'.format(y)] / df['영업이익_{}_12'.format(y-1)]) * 100) - 100
        df['당기순이익증가율_{}_12'.format(y)] = ((df['당기순이익_{}_12'.format(y)] / df['당기순이익_{}_12'.format(y-1)]) * 100) - 100
        df['자산총계증가율_{}_12'.format(y)] = ((df['자산_{}_12'.format(y)] / df['자산_{}_12'.format(y-1)]) * 100) - 100
    
        for k, j in enumerate(data['profit']):
            df1 = df[df['{}_{}_12'.format(j, y)] >= 수익성임계치[k][0]]
            df2 = df[(df['{}_{}_12'.format(j, y)] >= 수익성임계치[k][1]) & (df['{}_{}_12'.format(j, y)] < 수익성임계치[k][0])]
            df3 = df[(df['{}_{}_12'.format(j, y)] < 수익성임계치[k][1])]
            df1['{}_{}_12_점수'.format(j, y)] = 수익성점수[i][0]
            df2['{}_{}_12_점수'.format(j, y)] = 수익성점수[i][1]
            df3['{}_{}_12_점수'.format(j, y)] = 수익성점수[i][2]
            df = pd.concat([df1, df2, df3])

        for j in data['grow']:
            df1 = df[df['{}_{}_12'.format(j, y)] >= 10]
            df2 = df[(df['{}_{}_12'.format(j, y)] >= 4) & (df['{}_{}_12'.format(j, y)] < 10)]
            df3 = df[(df['{}_{}_12'.format(j, y)] < 4)]
            df1['{}_{}_12_점수'.format(j, y)] = 성장성점수[i][0]
            df2['{}_{}_12_점수'.format(j, y)] = 성장성점수[i][1]
            df3['{}_{}_12_점수'.format(j, y)] = 성장성점수[i][2]
            df = pd.concat([df1, df2, df3])

        for k, j in enumerate(data['stable'][:-1]):
            df1 = df[df['{}_{}_12'.format(j, y)] >= 200]
            df2 = df[(df['{}_{}_12'.format(j, y)] >= 100) & (df['{}_{}_12'.format(j, y)] < 200)]
            df3 = df[(df['{}_{}_12'.format(j, y)] < 100)]
            df1['{}_{}_12_점수'.format(j, y)] = 안정성점수1[k][i][0]
            df2['{}_{}_12_점수'.format(j, y)] = 안정성점수1[k][i][1]
            df3['{}_{}_12_점수'.format(j, y)] = 안정성점수1[k][i][2]
            df = pd.concat([df1, df2, df3])

        j = data['stable'][-1]
        df1 = df[df['{}_{}_12'.format(j, y)] >= 8]
        df2 = df[(df['{}_{}_12'.format(j, y)] < 8)]
        df1['{}_{}_12_점수'.format(j, y)] = 안정성점수2[i][0]
        df2['{}_{}_12_점수'.format(j, y)] = 안정성점수2[i][1]
        df = pd.concat([df1, df2])
        
    for i, x in enumerate(data['market']):
        df1 = df[df['{}_{}_12'.format(x, years[3])] > df['{}_{}_12'.format(x, years[2])]]
        df11 = df1[df1['{}_{}_12'.format(x, years[2])] > df1['{}_{}_12'.format(x, years[1])]] 
        df111 = df11[df11['{}_{}_12'.format(x, years[1])] > df11['{}_{}_12'.format(x, years[0])]] 
        df112 = df11[df11['{}_{}_12'.format(x, years[1])] <= df11['{}_{}_12'.format(x, years[0])]] 
        df12 = df1[df1['{}_{}_12'.format(x, years[2])] <= df1['{}_{}_12'.format(x, years[1])]] 
        df2 = df[df['{}_{}_12'.format(x, years[3])] <= df['{}_{}_12'.format(x, years[2])]] 

        df111['{}_{}_12_점수'.format(x, years[3])] = [시장가치비율점수[i][0]] * df111.shape[0]
        df112['{}_{}_12_점수'.format(x, years[3])] = [시장가치비율점수[i][1]] * df112.shape[0]
        df12['{}_{}_12_점수'.format(x, years[3])] = [시장가치비율점수[i][2]] * df12.shape[0]
        df2['{}_{}_12_점수'.format(x, years[3])] = [시장가치비율점수[i][3]] * df2.shape[0]
        df = pd.concat([df111, df112, df12, df2])
    
    for d in list(data.keys()):
        df['{}_score'.format(d)] = [0] * df.shape[0]
        for s in data[d]:
            for y in years[1:]:
                try:
                    df['{}_score'.format(d)] += df['{}_{}_12_점수'.format(s, y)]
                except:
                    pass
    return df

def mergeAnalyzeTable(data, df_list, filter_list, save_excel=True):
    df = df_list[0]
    filename = filter_list[0]
    for i, d in enumerate(df_list[1:]):
        df = pd.merge(df, d, on=['종목코드'])
        filename = filename + '_' + filter_list[i+1]
        data[filename] = df
        if save_excel:
            df.to_excel(filename+'.xlsx', index=False, encoding='utf-8-sig')
        try:
            print('{}: {}'.format(filename, df.shape[0]))
            df['수익률'] = ((df['최신가격'] - df['기준가격']) / df['기준가격']) * 100
            print('수익률: {}'.format(df['수익률'].mean()))
        except:
            pass
    return 

In [68]:
main = collector.Collector('dart')
main._setMaria()

# Compute

파라미터 설정

In [69]:
selected_year = 2020
selected_qt = '09'
std_date = '20201116'
res_tablename = 'value_result_{}_{}_new'.format(selected_year, selected_qt)

데이터 로드

In [70]:
main._setMaria()
data = {}
years = [selected_year-3, selected_year-2, selected_year-1, selected_year]

# 데이터 로드
loadTables(data, years, selected_qt, main)

# 컬럼 이름 바꾸기 ex) 자산총계 -> 자산총계_2017_12
modifyColNames(data)

# 데이터 병합해서 한 테이블로 만들기
df = mergeTables(data)

if selected_qt != '12':
    # 추정 당기순이익 계산
    df = calDg(df, years, selected_qt)
    
# EPS 계산
df = calEPS(df, years)

# 종가 데이터 추가
df = mergePrice(df, std_date)

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


계산

In [71]:
# 가치분석 계산하기
df = calEvaluation(df, years)
df = calScores(df)

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

디비 저장

In [72]:
# 결과 테이블
df = df.dropna()
main.maria.createTable(res_tablename, list(df.columns), ['VARCHAR(20)'] + ['FLOAT']*(len(df.columns)-1))
for i in range(df.shape[0]):
    try:
        main.maria.insertData(res_tablename, tuple(list(df.iloc[i])))
        main.maria.commitDB()
    except:
        continue

insert into value_result_2020_09_new values ('036620', 20400000000.0, 10800000000.0, 9600000000.0, 1300000000.0, 11100000000.0, 9600000000.0, 14800000000.0, 0.0, 3000000000.0, 0.0, 13873300000.0, 482933000.0, -3061250000.0, 69111400.0, 0.0, 0.0, 16500000000.0, 6000000000.0, 10400000000.0, 4600000000.0, 8000000000.0, 4200000000.0, 8900000000.0, -1900000000.0, 800000000.0, 300000000.0, 19728200000.0, 200335000.0, -6783530000.0, 69111400.0, 0.0, 0.0, 39200000000.0, 22900000000.0, 16300000000.0, 6600000000.0, 24500000000.0, 22500000000.0, 7400000000.0, -1400000000.0, 0.0, 0.0, 28410600000.0, 4008710000.0, -5950120000.0, 69111400.0, 0.0, 0.0, 310169000.0, 69111400.0, 0.0, 0.0, -573220000.0, 69111400.0, 0.0, 0.0, -297208000.0, 69111400.0, 0.0, 0.0, -2124780000.0, 69111400.0, 0.0, 0.0, 39200000000.0, 22900000000.0, 16300000000.0, 6600000000.0, 24500000000.0, 22500000000.0, 7400000000.0, -1400000000.0, 0.0, 0.0, 28410600000.0, 4008710000.0, 69111400.0, 0.0, 0.0, 16300000000.0, -0.4439, 3955641

insert into value_result_2020_09_new values ('208860', 11100000000.0, 1700000000.0, 9400000000.0, 1400000000.0, 4000000000.0, 1500000000.0, 3400000000.0, 0.0, 1000000000.0, 0.0, 1563590000.0, 1908120000.0, 15687600.0, 11443900.0, 0.0, 0.0, 31000000000.0, 20400000000.0, 10600000000.0, 7100000000.0, 18200000000.0, 20000000000.0, 4200000000.0, -900000000.0, 1000000000.0, 0.0, 4690770000.0, 2630400000.0, -962454000.0, 11443900.0, 0.0, 0.0, 86200000000.0, 60200000000.0, 25900000000.0, 9700000000.0, 41000000000.0, 41200000000.0, 6300000000.0, -3500000000.0, 3200000000.0, 5200000000.0, 5023280000.0, 14853900000.0, -8358890000.0, 11443900.0, 0.0, 0.0, 124466000.0, 11443900.0, 0.0, 0.0, -563806000.0, 11443900.0, 0.0, 0.0, -1803010000.0, 11443900.0, 0.0, 0.0, -1860730000.0, 11443900.0, 0.0, 0.0, 86200000000.0, 60200000000.0, 25900000000.0, 9700000000.0, 41000000000.0, 41200000000.0, 6300000000.0, -3500000000.0, 3200000000.0, 5200000000.0, 5023280000.0, 14853900000.0, 11443900.0, 0.0, 0.0, 960000

insert into value_result_2020_09_new values ('026040', 109900000000.0, 37800000000.0, 72000000000.0, 4000000000.0, 70700000000.0, 36300000000.0, 139900000000.0, 0.0, 26400000000.0, 1500000000.0, 8684790000.0, 4048510000.0, -3628730000.0, 16503800.0, 0.0, 672391.0, 115200000000.0, 43800000000.0, 71400000000.0, 9700000000.0, 71400000000.0, 42800000000.0, 127400000000.0, -900000000.0, 33000000000.0, 1000000000.0, 8684790000.0, 3164770000.0, 243541000.0, 16503800.0, 0.0, 672391.0, 82000000000.0, 36700000000.0, 45300000000.0, 3100000000.0, 40700000000.0, 34700000000.0, 96100000000.0, -28500000000.0, 26000000000.0, 500000000.0, 8684790000.0, 2853650000.0, -33441700000.0, 16503800.0, 0.0, 672391.0, 238515000.0, 16503800.0, 0.0, 672391.0, -1184630000.0, 16503800.0, 0.0, 672391.0, -4315780000.0, 16503800.0, 0.0, 672391.0, -3242030000.0, 16503800.0, 0.0, 672391.0, 82000000000.0, 36700000000.0, 45300000000.0, 3100000000.0, 40700000000.0, 34700000000.0, 96100000000.0, -28500000000.0, 26000000000.0

insert into value_result_2020_09_new values ('149980', 50000000000.0, 9800000000.0, 40200000000.0, 1700000000.0, 31500000000.0, 8800000000.0, 18100000000.0, 1700000000.0, 4300000000.0, 0.0, 1409470000.0, 1604540000.0, 762581000.0, 14100000.0, 0.0, 1766380.0, 43000000000.0, 4600000000.0, 38400000000.0, 2800000000.0, 25700000000.0, 3800000000.0, 20300000000.0, 0.0, 100000000.0, 0.0, 1409470000.0, 791342000.0, 140763000.0, 14100000.0, 0.0, 1766380.0, 46900000000.0, 5300000000.0, 41500000000.0, 2100000000.0, 29300000000.0, 4200000000.0, 20100000000.0, 2600000000.0, 1400000000.0, 0.0, 1410000000.0, 1104630000.0, 3048460000.0, 14100000.0, 0.0, 1766380.0, 506573000.0, 14100000.0, 0.0, 1766380.0, -394412000.0, 14100000.0, 0.0, 1766380.0, 1227560000.0, 14100000.0, 0.0, 1766380.0, 588784000.0, 14100000.0, 0.0, 1766380.0, 46900000000.0, 5300000000.0, 41500000000.0, 2100000000.0, 29300000000.0, 4200000000.0, 20100000000.0, 2600000000.0, 1400000000.0, 0.0, 1410000000.0, 1104630000.0, 14100000.0, 0.

# Analyze

파라미터 설정

In [73]:
today = '20210311'

조건 테이블 생성

In [86]:
'''
a. 기준가격 > 주당가치분석
b. 주당가치분석 < 기업가치
c. 영업이익 3년 연속 상승
d. 부채비율 n% 이하
e. 본질가치 > 주당기업가치
h. 현재가 < 적정주가
'''
a_df = main.analyzeData('a', res_tablename, std_date=std_date , today=today, selected_year=selected_year)
b_df = main.analyzeData('b', res_tablename, std_date=std_date , today=today, selected_year=selected_year)
c_df = main.analyzeData('c', res_tablename, std_date=std_date , today=today, selected_year=selected_year)
d_df1 = main.analyzeData('d', res_tablename, std_date=std_date , today=today, selected_year=selected_year, dept_rate=100)
d_df2 = main.analyzeData('d', res_tablename, std_date=std_date , today=today, selected_year=selected_year, dept_rate=200)
g_df = main.analyzeData('g', res_tablename, std_date=std_date , today=today, selected_year=selected_year)
h_df = main.analyzeData('h', res_tablename, std_date=std_date , today=today, selected_year=selected_year)
orgin_df = main.analyzeData('orgin', res_tablename, std_date=std_date , today=today, selected_year=selected_year)


필터링

In [87]:
data = {}
# mergeAnalyzeTable(data, [a_df, g_df], ['기준가격', '최신가격'])
# data['기준가격_최신가격']['수익률'] = ((data['기준가격_최신가격']['최신가격'] - data['기준가격_최신가격']['기준가격']) / data['기준가격_최신가격']['기준가격']) * 100

print('전체: {}'.format(orgin_df.shape[0]))

df_list = [a_df, c_df, d_df1, h_df]
filter_list = ['기준가격', '영업이익', '부채비율', '적정가격']
mergeAnalyzeTable(data, df_list, filter_list)


# comb_ac = pd.merge(a_df, c_df, on=['종목코드'])
# comb_ac.to_excel('영업이익(억)_{}.xlsx'.format(std_date), index=False, encoding='utf-8-sig')
# print('영업이익: {}'.format(comb_ac.shape[0]))
# print('수익률: {}'.format(comb_ac['수익률'].mean()))

# comb_ad1 = pd.merge(a_df, d_df1, on=['종목코드'])
# comb_ad1.to_excel('부채비율{}(억)_{}.xlsx'.format(100, std_date), index=False, encoding='utf-8-sig')
# print('부채비율{}: {}'.format(100, comb_ad1.shape[0]))
# print('수익률: {}'.format(comb_ad1['수익률'].mean()))

# comb_ad2 = pd.merge(a_df, d_df2, on=['종목코드'])
# comb_ad2.to_excel('부채비율{}(억)_{}.xlsx'.format(200, std_date), index=False, encoding='utf-8-sig')
# print('부채비율{}: {}'.format(200, comb_ad2.shape[0]))
# print('수익률: {}'.format(comb_ad2['수익률'].mean()))

# comb_acd1 = pd.merge(comb_ac, d_df1, on=['종목코드'])
# comb_acd1.to_excel('영업이익_부채비율{}(억)_{}.xlsx'.format(100, std_date), index=False, encoding='utf-8-sig')
# print('영업이익_부채비율{}: {}'.format(100, comb_acd1.shape[0]))
# print('수익률: {}'.format(comb_acd1['수익률'].mean()))

# comb_acd2 = pd.merge(comb_ac, d_df2, on=['종목코드'])
# comb_acd2.to_excel('영업이익_부채비율{}(억)_{}.xlsx'.format(200, std_date), index=False, encoding='utf-8-sig')
# print('영업이익_부채비율{}: {}'.format(200, comb_acd2.shape[0]))
# print('수익률: {}'.format(comb_acd2['수익률'].mean()))

# comb_acdh1 = pd.merge(comb_acd1, h_df, on=['종목코드'])
# comb_acdh1.to_excel('적정가격_영업이익_부채비율{}(억)_{}.xlsx'.format(100, std_date), index=False, encoding='utf-8-sig')
# print('적정가격_영업이익_부채비율{}: {}'.format(100, comb_acdh1.shape[0]))
# print('수익률: {}'.format(comb_acdh1['수익률'].mean()))

# comb_acdh2 = pd.merge(comb_acd2, h_df, on=['종목코드'])
# comb_acdh2.to_excel('적정가격_영업이익_부채비율{}(억)_{}.xlsx'.format(200, std_date), index=False, encoding='utf-8-sig')
# print('적정가격_영업이익_부채비율{}: {}'.format(200, comb_acdh2.shape[0]))
# print('수익률: {}'.format(comb_acdh2['수익률'].mean()))


price = []
rate = []
from_date_str = std_date[:4] + std_date[4:6] + std_date[6:]
# 코스피 종목수 934
df = fdr.DataReader('KS11', std_date, today)
df['rate'] = ((np.array(df['Close']) - df['Close'][from_date_str]) / df['Close'][from_date_str])*100
price.append(['KOSPI']+list(df['Close']))
rate.append(['KOSPI']+list(df['rate'])) 

# 코스닥 종목수 1,471
df = fdr.DataReader('KQ11', std_date, today)
df['rate'] = ((np.array(df['Close']) - df['Close'][from_date_str]) / df['Close'][from_date_str])*100
price.append(['KOSDAQ']+list(df['Close']))
rate.append(['KOSDAQ']+list(df['rate'])) 


print('KOSPI 상승률: {}'.format(rate[0][-1]))
print('KOSDAQ 상승률: {}'.format(rate[1][-1]))
print('가중평균 상승률: {}'.format((rate[0][-1]*934 + rate[1][-1]*1471) / (934+1471)))

score_tablename = 'value_score_{}_{}_new'.format(selected_year, selected_qt)
df = pd.merge(data['기준가격_영업이익_부채비율_적정가격'][['종목코드']], orgin_df, on=['종목코드'])
df = calScores(df)
main.maria.createTable(score_tablename, ['code'] + list(df.columns)[-5:] + ['min_band_price'], ['VARCHAR(20)'] + ['INT']*6)
for i in range(df.shape[0]):
    try:
        main.maria.insertData(score_tablename, tuple([df['종목코드'].iloc[i]] + list(df[list(df.columns)[-5:]].astype('int').iloc[i]) + [int(df['주당가치분석'].iloc[i])]))
        main.maria.commitDB()
    except:
        continue

전체: 1588
기준가격_영업이익: 107
기준가격_영업이익_부채비율: 61
기준가격_영업이익_부채비율_적정가격: 39
수익률: 14.979073942667947
KOSPI 상승률: 18.508236237873703
KOSDAQ 상승률: 7.161318494565275
가중평균 상승률: 11.567980104648464


C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\choi97201\anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

insert into value_score_2020_09_new values ('109860', 5, 92, 47, 100, 60, 20338);
(1062, "Duplicate entry '109860' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('051360', 5, 92, 37, 97, 60, 14096);
(1062, "Duplicate entry '051360' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('017890', 5, 82, 61, 100, 60, 14558);
(1062, "Duplicate entry '017890' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('111770', 5, 80, 59, 100, 60, 33692);
(1062, "Duplicate entry '111770' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('002170', 5, 86, 47, 100, 60, 106031);
(1062, "Duplicate entry '002170' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('009970', 5, 80, 55, 100, 60, 100592);
(1062, "Duplicate entry '009970' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('170900', 5, 78, 43, 97, 60, 88550);
(1062, "Duplicate entry '170900' for key 'PRIMARY'")
insert into value_score_2020_09_new values ('036530', 5

In [63]:
import cjw_maria
import pandas as pd

host = "localhost"
user = "root"
password = "sa1234"
db = "jamoo"
maria = cjw_maria.MariaDB(host, user, password, db)
score_df = maria.showData('select * from value_score_2020_09')

host = "15.165.29.213"
user = "lt_user"
password = "De4IjOY32e7o" 
db = "leaderstrading"
maria = cjw_maria.MariaDB(host, user, password, db)
maria.createTable('value_score_2020_09', list(score_df.columns), ['VARCHAR(20)']+['INT']*(score_df.shape[1]-1))
for i in range(score_df.shape[0]):
    maria.insertData('value_score_2020_09', tuple(list(score_df.iloc[i])))
    maria.commitDB()